In [1]:
# reading MicroManager Data

In [3]:
import tifffile as tff
from packaging import version
if version.parse(tff.__version__) < version.parse('2023.7.10'):
    raise ImportError(f"tifffile version {tff.__version__} is not supported.")

import zarr
import dask.array
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Union

import logging
logger=logging.getLogger()
# standard logging config, if you don't have Rich installed
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger.info(f"tifffile version: {tff.__version__}")

In [4]:
if not logger.hasHandlers():
    print("Make sure logging works!")
    print("Look for the next cell when you have Rich installed.")
else:
    logger.info("Welcome!")

Make sure logging works!
Look for the next cell when you have Rich installed.


In [5]:
# Rich can print beautiful and help debug:
# Install with pip install rich + pip install "rich[jupyter]"
from rich.logging import RichHandler
if logger.hasHandlers():
    logger.handlers.clear()
logging.basicConfig(level=logging.INFO, handlers=[RichHandler()])
logger.info("Welcome!")
logger.warning("Oh look up!")
logger.error("Is there something wrong?")
from rich import inspect

[04/08/24 09:59:24] INFO     INFO:root:Welcome!                                                     ]8;id=360693;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\4032528959.py\4032528959.py]8;;\:]8;id=416666;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\4032528959.py#7\7]8;;\

                    WARNING  WARNING:root:Oh look up!                                               ]8;id=282563;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\4032528959.py\4032528959.py]8;;\:]8;id=636220;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\4032528959.py#8\8]8;;\

                    ERROR    ERROR:root:Is there something wrong?                                   ]8;id=908372;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\4032528959.py\4032528959.py]8;;\:]8;id=516075;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\4032528959.py#9\9]8;;\

In [ ]:
from rich import print as rprint
rprint("[italic red]Hello[/italic red] World!", locals())

In [ ]:
from rich import inspect
inspect(tff, methods=True)

In [2]:
# In all examples the first image of the multifile tiff store hase to be passed.
# This examples should be wrapped in a file reade class,
# so that changes in package, file format or other changes don't influence your code
# This will be in a seperated follow up, which is not part of this notebook

In [3]:
# --------- Open MM ome.tiff or NDtiff multifile dataset lazy loaded as zarr.array using Tiffile and zarr ---------

In [6]:
# Open MicorManager ome.tiff dataset (multifile) with tiffile:
#image_path = "P://TestData/NDTiffExample_1/NDTiffExample_1_NDTiffStack.tif" #'/path/to/first/file_in_multi_fileND.tif'
image_path = Path("P://TestData/TiffWithPositionsDualCam_1/TiffWithPositionsDualCam_1_MMStack.ome.tif") #'/path/to/first/file_in_multi_file.ome.tif'
#image_path = Path("D://Temp/2024-02-06_12-11-45_cam4_1per_AVE_ctrl_R2_MMStack.ome.tif")
store = tff.TiffFile(image_path) # open tiffile store
if logger.level == logging.DEBUG:
    inspect(store)
logger.info(f"ís micromanager: {store.is_micromanager}")
logger.info(f"is MMStack: {store.is_mmstack}")
logger.info(f"is NDTiff: {store.is_ndtiff}")
logger.info(f"Info of dataset (all files): {store.series[0]}")
axes = store.series[0].axes
logger.info(f"Axes of dataset: {store.series[0].axes}")
logger.info(f"Shape of dataset: {store.series[0].shape}")
meteadata = store.micromanager_metadata

[04/08/24 09:59:28] INFO     INFO:root:ís micromanager: True                                         ]8;id=213306;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\593397945.py\593397945.py]8;;\:]8;id=502470;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\593397945.py#8\8]8;;\

                    INFO     INFO:root:is MMStack: True                                              ]8;id=341752;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\593397945.py\593397945.py]8;;\:]8;id=706504;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\593397945.py#9\9]8;;\

                    INFO     INFO:root:is NDTiff: False                                             ]8;id=426926;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\593397945.py\593397945.py]8;;\:]8;id=298474;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\593397945.py#10\10]8;;\

                    INFO     INFO:root:Info of dataset (all files): TiffPageSeries 0                ]8;id=462014;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\593397945.py\593397945.py]8;;\:]8;id=8551;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\593397945.py#11\11]8;;\
                             200x10x2x512x512  uint16  TZCYX  mmstack  4000 Pages                                  

                    INFO     INFO:root:Axes of dataset: TZCYX                                       ]8;id=524692;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\593397945.py\593397945.py]8;;\:]8;id=846502;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\593397945.py#13\13]8;;\

                    INFO     INFO:root:Shape of dataset: (200, 10, 2, 512, 512)                     ]8;id=9722;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\593397945.py\593397945.py]8;;\:]8;id=484756;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\593397945.py#14\14]8;;\

In [7]:
# lazy load ome.tiff dataset by mapping to zarr array usning the TiffFile store:
zarr_array = zarr.open(store.aszarr(), mode='r')
logger.info(f"Type: {zarr_array}")
logger.info(f"Dimensions: {zarr_array}")
last_image_in_last_file = zarr_array[-1,-1]
logger.info(f"Dimensions of last image: {last_image_in_last_file.shape}")

[04/08/24 09:59:32] INFO     INFO:root:Type: <zarr.core.Array (200, 10, 2, 512, 512) uint16         ]8;id=469675;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\1954770375.py\1954770375.py]8;;\:]8;id=502427;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\1954770375.py#3\3]8;;\
                             read-only>                                                                            

                    INFO     INFO:root:Dimensions: <zarr.core.Array (200, 10, 2, 512, 512) uint16   ]8;id=211565;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\1954770375.py\1954770375.py]8;;\:]8;id=429573;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\1954770375.py#4\4]8;;\
                             read-only>                                                                            

                    INFO     INFO:root:Dimensions of last image: (2, 512, 512)                      ]8;id=32376;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\1954770375.py\1954770375.py]8;;\:]8;id=85204;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\1954770375.py#6\6]8;;\

In [98]:
# ---------- Open MM ome.tiff or NDtiff multifile dataset lazy loaded as dask.array ----------

In [8]:
# lazy load ome.tiff dataset by mapping zarr_store to dask array:
# dask array doesen't proviede axis information
#image_path = "P://TestData/NDTiffExample_1/NDTiffExample_1_NDTiffStack.tif" #'/path/to/first/file_in_multi_file.ome.tif'
#store = tff.imread(image_path, aszarr=True) #also gives a zarr store like store.aszarr()
dask_array = dask.array.from_zarr(store.aszarr())
logger.info(f"Type: {dask_array}")
logger.info(f"Dimensions: {dask_array.shape}")
logger.info(f"Dimensions of last image: {dask_array[-1].shape}")

[04/08/24 09:59:37] INFO     INFO:root:Type: dask.array<from-zarr, shape=(200, 10, 2, 512, 512),     ]8;id=526451;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\695971279.py\695971279.py]8;;\:]8;id=609686;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\695971279.py#6\6]8;;\
                             dtype=uint16, chunksize=(1, 1, 1, 512, 512), chunktype=numpy.ndarray>                 

                    INFO     INFO:root:Dimensions: (200, 10, 2, 512, 512)                            ]8;id=291106;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\695971279.py\695971279.py]8;;\:]8;id=970801;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\695971279.py#7\7]8;;\

                    INFO     INFO:root:Dimensions of last image: (10, 2, 512, 512)                   ]8;id=195159;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\695971279.py\695971279.py]8;;\:]8;id=923479;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\695971279.py#8\8]8;;\

In [6]:
# --------- Open MM ndtiff multifile dataset lazy loaded as dask.array using ndtiff package ---------
# This is recomendet for maximum performance

In [9]:
from ndtiff import Dataset

In [10]:
#logger.level=logging.DEBUG
image_path = "P://TestData/NDTiffWithPositionsDualCam_1/" #'/path/to/dataset_folder/'
data = Dataset(image_path)
dask_array = data.as_array()
logger.info(f"Datastore: {data}")
logger.info(f"dask array: {dask_array}")
logger.info(f"dask array dimensions [position,time,channel,z,y,x]: {dask_array.shape}")

Dataset opened                 


[04/08/24 09:59:46] INFO     INFO:root:Datastore: <ndtiff.nd_tiff_current.NDTiffDataset object at   ]8;id=117158;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\3937123280.py\3937123280.py]8;;\:]8;id=528700;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\3937123280.py#5\5]8;;\
                             0x000001F64C66CD00>                                                                   

                    INFO     INFO:root:dask array: dask.array<_read_one_image, shape=(5, 938, 2,    ]8;id=257537;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\3937123280.py\3937123280.py]8;;\:]8;id=711911;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\3937123280.py#6\6]8;;\
                             10, 512, 512), dtype=uint16, chunksize=(1, 1, 1, 1, 512, 512),                        
                             chunktype=numpy.ndarray>                                                              

                    INFO     INFO:root:dask array dimensions [position,time,channel,z,y,x]: (5,     ]8;id=320511;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\3937123280.py\3937123280.py]8;;\:]8;id=72499;file://C:\Users\Lukas\AppData\Local\Temp\ipykernel_274584\3937123280.py#7\7]8;;\
                             938, 2, 10, 512, 512)                                                                 

In [11]:
# Read single image:
# data.read_image(channel= , z=, time=, position=, row=, column=)
# pass all four main indices: time, z, position, channel
# index starts at 0!
print(data.has_image(time=930,z=9,position=0,channel=0))
last_image_in_last_file = data.read_image(time=930,z=9,position=0,channel=0)
first_image = data.read_image(time=0,z=0,position=0,channel=0)
print(type(last_image_in_last_file))
print(last_image_in_last_file.shape)

True
<class 'numpy.ndarray'>
(512, 512)


In [9]:
# view dask.array with napari
axis = [position,time,channel,z,y,x]
import napari
napari.view_image(dask_array, multiscale=False, rgb=False, axis_labels=axis)

Viewer(camera=Camera(center=(0.0, 255.5, 255.5), zoom=0.6471927266725352, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(499.0, 4.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=4, ndisplay=2, last_used=3, range=((0.0, 1000.0, 1.0), (0.0, 10.0, 1.0), (0.0, 512.0, 1.0), (0.0, 512.0, 1.0)), current_step=(499, 4, 255, 255), order=(0, 1, 2, 3), axis_labels=('time', 'z', 'y', 'x')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'dask_array' at 0x1ed6df3b8b0>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x000001ED628C1A60>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap={})

In [ ]:
# view microscopy data multichannel
# not tested yet
dask_array = [dask_array_ch1, dask_array_ch2]
# for ndtiff package arrays:
dask.array.concatenate(dask_array, axis=2) # axis=2 for ndtiff package arrays
# for tiffile arrays
dask.array.stack(dask_array, axis=1) # axis=1 is created, generating [t,c,z,x,y]

napari.view_image(dask_array, multiscale=False, rgb=False)

In [ ]:
# -------------- Explore Funciton for the Microscopy Data Reader Class -------------------

In [12]:
# get axis order
axis_order = "RTCZYX"
org_axis_position = [11,12,13,14,15,16]
print(org_axis_position[0])
for i, org in enumerate(axis_order):
    print(i, end=', ')
    print(org, end=', ')
    print(axes.find(org))
    org_axis_position[i] = axes.find(org)
print(org_axis_position)
added = 0
for i in range(len(org_axis_position)):
    print(i, end=', ')
    print(org_axis_position[i], end=': ') if org_axis_position[i] == -1 else print(org_axis_position[i]+added, end=': ')
    if org_axis_position[i] == -1:
        print(f'create new axis {axis_order[i]}')
        org_axis_position[i] = i
        org_axis_position[i+1:] = [x+1 if x >= 0 else x for x in org_axis_position[i+1:]]
    elif not org_axis_position[i]+added == i:
        print(f'swapp axis {org_axis_position[i]+added} with {i}')
        old_axis = org_axis_position[i]
        org_axis_position[i] = i
        org_axis_position[old_axis+added] = old_axis
    else:
        print("nothing todo")
print(org_axis_position)

11
0, R, -1
1, T, 0
2, C, 2
3, Z, 1
4, Y, 3
5, X, 4
[-1, 0, 2, 1, 3, 4]
0, -1: create new axis R
1, 1: nothing todo
2, 3: swapp axis 3 with 2
3, 3: nothing todo
4, 4: nothing todo
5, 5: nothing todo
[0, 1, 2, 3, 4, 5]


In [13]:
first_image = dask_array[0,0,0].compute()
print(first_image)

[[[5093 5048 5002 ... 5222 5180 5138]
  [5088 5043 4996 ... 5217 5175 5132]
  [5082 5037 4990 ... 5212 5170 5127]
  ...
  [1476 1431 1388 ... 1616 1568 1521]
  [1470 1426 1383 ... 1610 1562 1516]
  [1465 1420 1377 ... 1604 1556 1510]]

 [[2822 2768 2713 ... 2985 2931 2876]
  [2815 2761 2707 ... 2978 2924 2869]
  [2808 2754 2700 ... 2971 2917 2863]
  ...
  [ 608  608  610 ...  616  612  609]
  [ 608  609  610 ...  615  612  609]
  [ 608  609  611 ...  615  611  609]]

 [[1138 1104 1070 ... 1246 1210 1173]
  [1134 1100 1066 ... 1242 1205 1169]
  [1130 1095 1062 ... 1237 1200 1165]
  ...
  [1125 1160 1196 ... 1026 1058 1091]
  [1130 1165 1200 ... 1030 1062 1095]
  [1134 1169 1205 ... 1034 1066 1100]]

 ...

 [[3415 3414 3413 ... 3411 3413 3414]
  [3415 3414 3413 ... 3411 3413 3414]
  [3415 3414 3412 ... 3411 3413 3414]
  ...
  [1927 1890 1854 ... 2038 2001 1964]
  [1923 1886 1849 ... 2033 1996 1960]
  [1918 1881 1844 ... 2028 1992 1955]]

 [[2855 2830 2804 ... 2927 2904 2880]
  [2852 2826

In [14]:
# fix axis order
print(dask_array.shape)
axis_order = "RTCZYX"
org_axis_position = [11,12,13,14,15,16]
print(org_axis_position[0])
for i, org in enumerate(axis_order):
    print(i, end=', ')
    print(org, end=', ')
    print(axes.find(org))
    org_axis_position[i] = axes.find(org)
print(org_axis_position)
added = 0
for i in range(len(org_axis_position)):
    print(i, end=', ')
    print(org_axis_position[i], end=': ') if org_axis_position[i] == -1 else print(org_axis_position[i]+added, end=': ')
    if org_axis_position[i] == -1:
        print(f'create new axis {axis_order[i]}')
        dask_array = dask.array.expand_dims(dask_array, i)
        org_axis_position[i] = i
        org_axis_position[i+1:] = [x+1 if x >= 0 else x for x in org_axis_position[i+1:]]
    elif not org_axis_position[i] == i:
        print(f'swapp axis {org_axis_position[i]} with {i}')
        dask_array = dask.array.moveaxis(dask_array, org_axis_position[i], i)
        #print(dask.array.moveaxis(dask_array, org_axis_position[i], i).shape)
        old_axis = org_axis_position[i]
        org_axis_position[i] = i
        org_axis_position[old_axis] = old_axis
    else:
        org_axis_position[i] = i
        print("nothing todo")
print(org_axis_position)
print(dask_array.shape)

(5, 938, 2, 10, 512, 512)
11
0, R, -1
1, T, 0
2, C, 2
3, Z, 1
4, Y, 3
5, X, 4
[-1, 0, 2, 1, 3, 4]
0, -1: create new axis R
1, 1: nothing todo
2, 3: swapp axis 3 with 2
3, 3: nothing todo
4, 4: nothing todo
5, 5: nothing todo
[0, 1, 2, 3, 4, 5]
(1, 5, 2, 938, 10, 512, 512)


In [15]:
new_first_image = dask_array[0,0,0].compute()
if (new_first_image == first_image).all:
    print("works!")

works!
